In [2]:
from __future__ import print_function
import random
import os
import sys
import time
import numpy as np
import pandas as pd
import multiprocessing as mp
from deap import tools
from deap import base, creator
from sklearn.cluster import KMeans
from multiprocessing import Pool
from six.moves.urllib.request import urlretrieve
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Activation,AlphaDropout
from keras.optimizers import adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

/home/blackrose/anaconda3/envs/datasciencelab/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data_root = 'data'
german_data = os.path.join(data_root,'GermanCreditInput.xls')
german_label = os.path.join(data_root,'GermanCreditOutputClass1columnknn.xls')
australian_dataset = os.path.join(data_root,'australian dataset.xlsx')

In [4]:
nn_config = {
    'train_cycles':600,
}

In [5]:
x_data = np.array(pd.read_excel(german_data,header=None))
y_data = np.array(pd.read_excel(german_label,header=None))
print('Dataset Shape : {}\nDataset Labels Shape : {}'.format(x_data.shape,y_data.shape))

Dataset Shape : (1000, 30)
Dataset Labels Shape : (1000, 1)


In [6]:
mx = MinMaxScaler()
mx.fit(x_data)
p_x_data = mx.transform(x_data)

/home/blackrose/anaconda3/envs/datasciencelab/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
def data_generator(data,labels,batch_size) :
    class_data = [[],[]]
    for i,row in enumerate(data) :
        if labels[i] == 0 :
            class_data[0].append(list(row))
        else :
            class_data[1].append(list(row))
    while True :
        batch = []
        batch_label = []
        for _ in range(batch_size) :
            choice = 1 if np.random.random() > .5 else 0
            batch_label.append(choice)
            batch.append(class_data[choice][np.random.randint(len(class_data[choice]))])
        yield (np.array(batch),np.array(batch_label))     

In [20]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
skf = StratifiedKFold(n_splits=10)
sum_val_acc = 0
for train_index, test_index in skf.split(p_x_data, y_data):
    model = Sequential()
    model.add(Dense(100, input_dim=30, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(100, kernel_initializer='glorot_normal', activation='selu'))
    model.add(Dense(100, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(100,  kernel_initializer='glorot_normal', activation='selu'))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
    hist = model.fit_generator(data_generator(p_x_data[train_index,:],y_data[train_index,:],32),epochs=int(10),steps_per_epoch=40,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    sum_val_acc += hist.history['val_acc'].pop()
    del model
print(sum_val_acc/10)

Epoch 1/10
40/40 [==============================] - 3s 87ms/step - loss: 0.7204 - acc: 0.5797 - val_loss: 0.4762 - val_acc: 0.8000
Epoch 2/10
40/40 [==============================] - 0s 7ms/step - loss: 0.6692 - acc: 0.6070 - val_loss: 0.4913 - val_acc: 0.8000
Epoch 3/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6334 - acc: 0.6398 - val_loss: 0.5060 - val_acc: 0.8000
Epoch 4/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6137 - acc: 0.6703 - val_loss: 0.5282 - val_acc: 0.7400
Epoch 5/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6048 - acc: 0.6633 - val_loss: 0.4799 - val_acc: 0.7500
Epoch 6/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6199 - acc: 0.6625 - val_loss: 0.4765 - val_acc: 0.7800
Epoch 7/10
40/40 [==============================] - 0s 7ms/step - loss: 0.6004 - acc: 0.6773 - val_loss: 0.4548 - val_acc: 0.8000
Epoch 8/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6032 - acc: 0.6

Epoch 4/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5949 - acc: 0.6914 - val_loss: 0.6428 - val_acc: 0.6900
Epoch 5/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5965 - acc: 0.6883 - val_loss: 0.6263 - val_acc: 0.6700
Epoch 6/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5966 - acc: 0.6922 - val_loss: 0.6600 - val_acc: 0.6700
Epoch 7/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5783 - acc: 0.7063 - val_loss: 0.6971 - val_acc: 0.6500
Epoch 8/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5941 - acc: 0.6953 - val_loss: 0.6756 - val_acc: 0.6900
Epoch 9/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5507 - acc: 0.7281 - val_loss: 0.8671 - val_acc: 0.6500
Epoch 10/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5633 - acc: 0.7242 - val_loss: 0.6511 - val_acc: 0.6700
Epoch 1/10
40/40 [==============================] - 4s 94ms/step - loss: 0.7556 - acc: 0.

In [19]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
skf = StratifiedKFold(n_splits=10)
sum_val_acc = 0
for train_index, test_index in skf.split(p_x_data, y_data):
    model = Sequential()
    model.add(Dense(100, input_dim=30, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(75, kernel_initializer='glorot_normal', activation='selu'))
    model.add(Dense(50, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(25,  kernel_initializer='glorot_normal', activation='selu'))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
    hist = model.fit_generator(data_generator(p_x_data[train_index,:],y_data[train_index,:],32),epochs=int(10),steps_per_epoch=40,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    sum_val_acc += hist.history['val_acc'].pop()
    del model
print(sum_val_acc/10)

Epoch 1/10
40/40 [==============================] - 3s 72ms/step - loss: 0.7991 - acc: 0.5437 - val_loss: 0.5395 - val_acc: 0.7800
Epoch 2/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6843 - acc: 0.5953 - val_loss: 0.5185 - val_acc: 0.7800
Epoch 3/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6624 - acc: 0.6211 - val_loss: 0.4848 - val_acc: 0.7800
Epoch 4/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6243 - acc: 0.6539 - val_loss: 0.4926 - val_acc: 0.7600
Epoch 5/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6161 - acc: 0.6680 - val_loss: 0.6205 - val_acc: 0.7100
Epoch 6/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6251 - acc: 0.6641 - val_loss: 0.5104 - val_acc: 0.7700
Epoch 7/10
40/40 [==============================] - 0s 6ms/step - loss: 0.5937 - acc: 0.6945 - val_loss: 0.5286 - val_acc: 0.7800
Epoch 8/10
40/40 [==============================] - 0s 6ms/step - loss: 0.5931 - acc: 0.6

Epoch 4/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6405 - acc: 0.6445 - val_loss: 0.5626 - val_acc: 0.7200
Epoch 5/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6058 - acc: 0.6797 - val_loss: 0.5869 - val_acc: 0.7300
Epoch 6/10
40/40 [==============================] - 0s 6ms/step - loss: 0.5992 - acc: 0.6828 - val_loss: 0.7230 - val_acc: 0.6600
Epoch 7/10
40/40 [==============================] - 0s 7ms/step - loss: 0.6017 - acc: 0.6914 - val_loss: 0.6810 - val_acc: 0.6700
Epoch 8/10
40/40 [==============================] - 0s 6ms/step - loss: 0.5676 - acc: 0.7047 - val_loss: 0.7377 - val_acc: 0.6400
Epoch 9/10
40/40 [==============================] - 0s 7ms/step - loss: 0.5760 - acc: 0.7070 - val_loss: 0.6149 - val_acc: 0.7000
Epoch 10/10
40/40 [==============================] - ETA: 0s - loss: 0.5716 - acc: 0.701 - 0s 6ms/step - loss: 0.5699 - acc: 0.7031 - val_loss: 0.6233 - val_acc: 0.6900
Epoch 1/10
40/40 [==============================] -

In [8]:
sum_val_acc = 0
skf = StratifiedKFold(n_splits=10)
sgd = SGD(lr=0.3, momentum=0.7)
for train_index, test_index in skf.split(x_data, y_data):
    model = Sequential()
    model.add(Dense(60, input_dim=30, activation='sigmoid'))
    model.add(Dense(60, input_dim=30, activation='sigmoid'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
    hist = model.fit_generator(data_generator(p_x_data[train_index,:],y_data[train_index,:],32),epochs=int(10),steps_per_epoch=40,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    sum_val_acc += hist.history['val_acc'].pop()
    del model
print(sum_val_acc/10)

Epoch 1/10
40/40 [==============================] - 0s 9ms/step - loss: 0.7149 - acc: 0.5258 - val_loss: 0.7157 - val_acc: 0.3000
Epoch 2/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6860 - acc: 0.5453 - val_loss: 0.6619 - val_acc: 0.7100
Epoch 3/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6763 - acc: 0.6078 - val_loss: 0.7224 - val_acc: 0.3100
Epoch 4/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6688 - acc: 0.6094 - val_loss: 0.6768 - val_acc: 0.6400
Epoch 5/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6570 - acc: 0.6703 - val_loss: 0.6689 - val_acc: 0.6400
Epoch 6/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6301 - acc: 0.6773 - val_loss: 0.6062 - val_acc: 0.7000
Epoch 7/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6137 - acc: 0.7055 - val_loss: 0.5975 - val_acc: 0.6900
Epoch 8/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5837 - acc: 0.73

Epoch 4/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6559 - acc: 0.6281 - val_loss: 0.6574 - val_acc: 0.7200
Epoch 5/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6396 - acc: 0.6984 - val_loss: 0.6736 - val_acc: 0.6200
Epoch 6/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6166 - acc: 0.7016 - val_loss: 0.6168 - val_acc: 0.7100
Epoch 7/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5759 - acc: 0.7328 - val_loss: 0.6218 - val_acc: 0.6800
Epoch 8/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5674 - acc: 0.7164 - val_loss: 0.6236 - val_acc: 0.6700
Epoch 9/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5371 - acc: 0.7375 - val_loss: 0.6156 - val_acc: 0.6800
Epoch 10/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5476 - acc: 0.7344 - val_loss: 0.5961 - val_acc: 0.6900
Epoch 1/10
40/40 [==============================] - 1s 13ms/step - loss: 0.7141 - acc: 0.

In [10]:
sum_val_acc = 0
for train_index, test_index in skf.split(p_x_data, y_data):
    model = Sequential()
    model.add(Dense(100, input_dim=30, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(100, kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(100,  kernel_initializer='glorot_normal', activation='selu'))
    model.add(AlphaDropout(.2))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
    hist = model.fit_generator(data_generator(p_x_data[train_index,:],y_data[train_index,:],32),epochs=int(10),steps_per_epoch=40,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    sum_val_acc += hist.history['val_acc'].pop()
    del model

Epoch 1/10
40/40 [==============================] - 1s 30ms/step - loss: 0.8124 - acc: 0.5437 - val_loss: 0.5672 - val_acc: 0.7200
Epoch 2/10
40/40 [==============================] - 0s 5ms/step - loss: 0.7676 - acc: 0.5492 - val_loss: 0.5339 - val_acc: 0.7400
Epoch 3/10
40/40 [==============================] - 0s 5ms/step - loss: 0.7314 - acc: 0.5789 - val_loss: 0.5591 - val_acc: 0.7000
Epoch 4/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6649 - acc: 0.6281 - val_loss: 0.5290 - val_acc: 0.7300
Epoch 5/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6639 - acc: 0.6383 - val_loss: 0.5501 - val_acc: 0.7300
Epoch 6/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6582 - acc: 0.6359 - val_loss: 0.5911 - val_acc: 0.7200
Epoch 7/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6439 - acc: 0.6469 - val_loss: 0.5300 - val_acc: 0.7600
Epoch 8/10
40/40 [==============================] - 0s 5ms/step - loss: 0.5915 - acc: 0.6

Epoch 4/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6682 - acc: 0.6313 - val_loss: 0.6754 - val_acc: 0.6700
Epoch 5/10
40/40 [==============================] - 0s 6ms/step - loss: 0.6532 - acc: 0.6547 - val_loss: 0.8328 - val_acc: 0.6400
Epoch 6/10
40/40 [==============================] - 0s 5ms/step - loss: 0.5939 - acc: 0.6969 - val_loss: 0.7077 - val_acc: 0.7400
Epoch 7/10
40/40 [==============================] - 0s 5ms/step - loss: 0.6206 - acc: 0.6844 - val_loss: 0.6848 - val_acc: 0.7200
Epoch 8/10
40/40 [==============================] - 0s 5ms/step - loss: 0.5846 - acc: 0.6977 - val_loss: 0.7230 - val_acc: 0.7200
Epoch 9/10
40/40 [==============================] - 0s 5ms/step - loss: 0.5954 - acc: 0.6938 - val_loss: 0.7516 - val_acc: 0.6700
Epoch 10/10
40/40 [==============================] - 0s 5ms/step - loss: 0.5972 - acc: 0.7055 - val_loss: 0.7024 - val_acc: 0.6800
Epoch 1/10
40/40 [==============================] - 2s 43ms/step - loss: 0.8135 - acc: 0.